<a href="https://colab.research.google.com/github/dvschultz/ai/blob/master/SinGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SinGAN
SinGAN is a ML library that allows you to do a lot of different things with one image as your dataset. As we’ve previously discussed usually you need a ton of images to be able to do anything in ML, so SinGAN needing only one image is a huge change.

It is, however, important to keep in mind what its outputs are. SinGAN, to me, is most like Photoshop’s Context Aware tools. It allows you to manipulate the single image in interesting ways, but in the end is only going to return you that same image, slightly modified. SinGAN is a bit of a Swiss Army Knife and that makes it pretty cool in a world of single-use ML tools.

Let’s take a look.

##Set up our Runtime
Colab needs to know we need to use a GPU-powered machine in order to do style transfers. At the top of this page, click on the `Runtime` tab, then select `Change runtime type`. In the modal that pops up, select `GPU` under the `Hardware accelerator` options.

We then need to make sure we’re using the latest version of Tensorflow 1, otherwise we get some annoying messages.

In [1]:
# use tensorflow 1.15 to avoid some issues
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


## Set up SinGAN
Let’s install the SinGAN repo and install all the dependencies needed

In [2]:
!git clone https://github.com/dvschultz/SinGAN
!python -m pip install -r /content/SinGAN/requirements.txt
%cd /content/SinGAN/

Cloning into 'SinGAN'...
remote: Enumerating objects: 849, done.
remote: Total 849 (delta 0), reused 0 (delta 0), pack-reused 849
Receiving objects: 100% (849/849), 94.90 MiB | 59.65 MiB/s, done.
Resolving deltas: 100% (350/350), done.
/content/SinGAN


We now have SinGAN installed on CoLab. Here are some relevant links to understand how to use the library:

*   [SinGAN Github repo](https://github.com/tamarott/SinGAN)
*   [ArXiv paper](https://arxiv.org/pdf/1905.01164.pdf)
* [Supplementary Materials paper](https://tomer.net.technion.ac.il/files/2019/09/SingleImageGan_SM.pdf)





##Training your image
In order to use SinGAN, we need to train the model on our image. In my experience this usually takes an hour or so to train a 250px image.

The `main_train.py` script takes one required argument, `--input_name`. To use with a custom image, upload an image to the `Input/Images` folder and then pass its name and file extension to the argument. 

In [3]:
!python main_train.py --input_name bernini.png --max_size 480

Random Seed:  7952
GeneratorConcatSkip2CleanAdd(
  (head): ConvBlock(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (body): Sequential(
    (block1): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block2): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block3): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

##Generating Random Samples
By default, once training is completed 50 samples are generated. That means you don’t need to run the next line of code, unless you want to change some of the default options. 

Note: If you do run the next line of code, you’ll need to delete the folder of output images that were created during training. Otherwise you will get an error.

###Sample Options

####Number of Images
`--num_samples` (note: this only works in my version of the library)

You can pass in a number to generate any number of images. By default it generates 50. I recommend using this when running the `main_train.py` script otherwise you’ll need to delete the output folder and re-run it.

####Generator scale
`--gen_start_scale` (pass is a number, usually between 0–3)

I don’t want to dig into this too deeply, so I’ll just say the generator scale relates to how "realistic" the new images look. If you use `0` you will get images that differ drastically from your original image. If you use `2` or `3` the differences will be very subtle. Anything above these numbers will give you almost no changes.

####Mode
`--mode`
There are two. While the default `random_samples` generates images of the same size as your training image, `random_samples_arbitrary_sizes` changes the size of your canvas. It’s not a scaled up image, however, its more like it makes the canvas larger and tries to fill the space added with a similar texture.

####Scale of canvas
`--scale_h` (pass it a floating number)

`--scale_v` 

For use with `random_samples_arbitrary_sizes`, this allows you to define how much larger (or smaller) the canvas should be. you should pass it a floating value that scales each dimension up or down (`1.0` is the same size, `2.0` is twice as large, etc).

In [23]:
# example using the default random_samples mode
!python random_samples.py --input_name bernini.png --mode random_samples --gen_start_scale 2 --num_samples 400 --max_size 720

Random Seed:  8234


In [26]:
# example using the random_samples_arbitrary_sizes mode
# can only run at --gen_start_scale 0 (https://github.com/tamarott/SinGAN/issues/61)
!python random_samples.py --input_name bernini.png --mode random_samples_arbitrary_sizes --scale_h 2.0 --scale_v 1.0 --gen_start_scale 2 --num_samples 200

Random Seed:  1223
Traceback (most recent call last):
  File "random_samples.py", line 50, in <module>
    SinGAN_generate(Gs, Zs, reals, NoiseAmp, opt, in_s, scale_v=opt.scale_v, scale_h=opt.scale_h)
  File "/content/SinGAN/SinGAN/manipulate.py", line 138, in SinGAN_generate
    z_in = noise_amp*(z_curr)+I_prev
RuntimeError: The size of tensor a (78) must match the size of tensor b (120) at non-singleton dimension 3


##Generating Animations
SinGAN can generate an animation from the single image. It essentially create a noisy peturbation on the image causing it to animate.

Unfortunately, this requires another training session on the image. It should take the same amount of time as your previous training.

`--min_size` and `--max_size` can be used here as well.

In [0]:
!python animation.py --input_name colusseum.png --max_size 32

Random Seed:  1816


###Animation Options

####Output type
`--output_type`

This allows you to output either `video` or `gif`. The default is `video`.

`--num_frames`

This determines the length of your animation. the default is `100` frames (10fps, so 10 seconds)

In [0]:
!python animation.py --input_name colusseum.png --max_size 32 --output_type gif --num_frames 50

In [27]:
!zip -r bernini-sq-scale2.zip /content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2

  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/ (stored 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/138.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/253.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/79.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/381.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/262.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/370.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/91.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/248.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2/264.png (deflated 0%)
  adding: content/SinGAN/Output/RandomSamples/bernini/gen_start_scale=2

In [0]:
rm -r /content/SinGAN/Output/RandomSamples_ArbitrerySizes/bernini/scale_v=1.000000_scale_h=1.500000

In [0]:
rm -r /content/SinGAN/Output/Animation/colusseum